In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
print("Setup Done")

# Mô tả bài toán

Cho các dữ liệu về các câu hỏi trên Quora dạng text và phân loại của chúng(Có phải toxic hay không?)

Đánh giá xem dữ liệu cho có phải là câu hỏi toxic hay không?

In [ ]:
raw_train_data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
raw_test_data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
#đọc file dữ liệu
raw_train_data.head()

In [ ]:
raw_test_data.head()

# Tiền xử lý dữ liệu
> # -Các công việc cần thiết
> Trước tiên,cần xử lý qua dữ liệu text này trước.Những việc cần phải làm những việc sau:
> > * **Sơ chế dữ liệu thô.**
> > * **Chia tập train và tập validation.**
> > * **Loại bỏ các nhiễu.**
> > * **Tạo vector feature cho dữ liệu**

****

> # -Sơ chế dữ liệu thô
> 
> Có 3 cột ở đây
> > * Id of question(Text)
> > * Question text(Text)
> > * Is question is sincere or not?(0/1)

In [ ]:
feature_name = ['question_text','target']
#lấy dữ liệu ở 2 cột question_text và target
train_data = raw_train_data[feature_name]
#tạo ra mảng mới có 2 cột dữ liệu question_text và target
print(train_data.head())
#in kết quả ra theo định dạng

> # -Chia tập train và tập validation
> * Phải tách tập train và tập validation ra sớm,tránh tập validation có thể bị ô nhiễm bởi tập train trong quá trình tiền xử lý dữ liệu

In [ ]:
### chia ra thành 2 tập,tập train và tập validation
train,val=train_test_split(train_data,test_size=0.2,stratify=train_data.target,random_state=123)
#Random 80% là mảng train và 20% là mảng validation
print("Shape of the Training set :",train.shape)
#in ra kích thức của mảng dữ liệu Train
print("Shape of the Validation set :",val.shape)
#in ra kích thức của mảng dữ liệu Validation
print(train.head())
#in dữ liệu ra theo định dạng của mảng Train
print(train.question_text[224356])
#in dữ liệu của question có id là 224356

* Test thử dữ liệu vừa tách

In [ ]:
#print(train.columns)
#print(train.question_text[602217])
#for i in range(0,10,1):
#    randIndex = random.randrange(0,train.question_text.size,1)
#   print(train.question_text[randIndex])


 phải reset_index cho dữ liệu mới để đồng bộ.

In [ ]:
 #sắp xếp lại các cột chỉ số
train = train.reset_index()
val = val.reset_index()
#in ra 10 gtri test thử
print(train.head(10))

> # -Loại bỏ các nhiễu
> Khi xử lý dữ liệu,cần lọc ra những dữ liệu cần thiết và không cần thiết cho việc dự đoán đánh giá của model.Trong bài này,dữ liệu cần là các từ,nên các kí tự đặc biệt và số sẽ bị loại bỏ.
> > * Loại bỏ các từ viết tắt
> > * Loại bỏ các kí tự đặc biệt,không có trong bảng chữ cái

* **Loại bỏ các từ viết tắt**

In [ ]:
#Loại bỏ các từ viết tắt 

#Tạo một directory có key là các từ viết tắt và value là các từ không viết tắt của key
#Mục đích để thay thế các từ viết tắt bằng các từ chuẩn,để chuẩn bị tạo danh sách từ vựng
contractions={"I'm": 'I am',
 "I'm'a": 'I am about to',
 "I'm'o": 'I am going to',
 "I've": 'I have',
 "I'll": 'I will',
 "I'll've": 'I will have',
 "I'd": 'I would',
 "I'd've": 'I would have',
 'Whatcha': 'What are you',
 "amn't": 'am not',
 "ain't": 'are not',
 "aren't": 'are not',
 "'cause": 'because',
 "can't": 'can not',
 "can't've": 'can not have',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "daren't": 'dare not',
 "daresn't": 'dare not',
 "dasn't": 'dare not',
 "didn't": 'did not',
 'didn’t': 'did not',
 "don't": 'do not',
 'don’t': 'do not',
 "doesn't": 'does not',
 "e'er": 'ever',
 "everyone's": 'everyone is',
 'finna': 'fixing to',
 'gimme': 'give me',
 "gon't": 'go not',
 'gonna': 'going to',
 'gotta': 'got to',
 "hadn't": 'had not',
 "hadn't've": 'had not have',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he've": 'he have',
 "he's": 'he is',
 "he'll": 'he will',
 "he'll've": 'he will have',
 "he'd": 'he would',
 "he'd've": 'he would have',
 "here's": 'here is',
 "how're": 'how are',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how's": 'how is',
 "how'll": 'how will',
 "isn't": 'is not',
 "it's": 'it is',
 "'tis": 'it is',
 "'twas": 'it was',
 "it'll": 'it will',
 "it'll've": 'it will have',
 "it'd": 'it would',
 "it'd've": 'it would have',
 'kinda': 'kind of',
 "let's": 'let us',
 'luv': 'love',
 "ma'am": 'madam',
 "may've": 'may have',
 "mayn't": 'may not',
 "might've": 'might have',
 "mightn't": 'might not',
 "mightn't've": 'might not have',
 "must've": 'must have',
 "mustn't": 'must not',
 "mustn't've": 'must not have',
 "needn't": 'need not',
 "needn't've": 'need not have',
 "ne'er": 'never',
 "o'": 'of',
 "o'clock": 'of the clock',
 "ol'": 'old',
 "oughtn't": 'ought not',
 "oughtn't've": 'ought not have',
 "o'er": 'over',
 "shan't": 'shall not',
 "sha'n't": 'shall not',
 "shalln't": 'shall not',
 "shan't've": 'shall not have',
 "she's": 'she is',
 "she'll": 'she will',
 "she'd": 'she would',
 "she'd've": 'she would have',
 "should've": 'should have',
 "shouldn't": 'should not',
 "shouldn't've": 'should not have',
 "so've": 'so have',
 "so's": 'so is',
 "somebody's": 'somebody is',
 "someone's": 'someone is',
 "something's": 'something is',
 'sux': 'sucks',
 "that're": 'that are',
 "that's": 'that is',
 "that'll": 'that will',
 "that'd": 'that would',
 "that'd've": 'that would have',
 'em': 'them',
 "there're": 'there are',
 "there's": 'there is',
 "there'll": 'there will',
 "there'd": 'there would',
 "there'd've": 'there would have',
 "these're": 'these are',
 "they're": 'they are',
 "they've": 'they have',
 "they'll": 'they will',
 "they'll've": 'they will have',
 "they'd": 'they would',
 "they'd've": 'they would have',
 "this's": 'this is',
 "those're": 'those are',
 "to've": 'to have',
 'wanna': 'want to',
 "wasn't": 'was not',
 "we're": 'we are',
 "we've": 'we have',
 "we'll": 'we will',
 "we'll've": 'we will have',
 "we'd": 'we would',
 "we'd've": 'we would have',
 "weren't": 'were not',
 "what're": 'what are',
 "what'd": 'what did',
 "what've": 'what have',
 "what's": 'what is',
 "what'll": 'what will',
 "what'll've": 'what will have',
 "when've": 'when have',
 "when's": 'when is',
 "where're": 'where are',
 "where'd": 'where did',
 "where've": 'where have',
 "where's": 'where is',
 "which's": 'which is',
 "who're": 'who are',
 "who've": 'who have',
 "who's": 'who is',
 "who'll": 'who will',
 "who'll've": 'who will have',
 "who'd": 'who would',
 "who'd've": 'who would have',
 "why're": 'why are',
 "why'd": 'why did',
 "why've": 'why have',
 "why's": 'why is',
 "will've": 'will have',
 "won't": 'will not',
 "won't've": 'will not have',
 "would've": 'would have',
 "wouldn't": 'would not',
 "wouldn't've": 'would not have',
 "y'all": 'you all',
 "y'all're": 'you all are',
 "y'all've": 'you all have',
 "y'all'd": 'you all would',
 "y'all'd've": 'you all would have',
 "you're": 'you are',
 "you've": 'you have',
 "you'll've": 'you shall have',
 "you'll": 'you will',
 "you'd": 'you would',
 "you'd've": 'you would have',
 'jan.': 'january',
 'feb.': 'february',
 'mar.': 'march',
 'apr.': 'april',
 'jun.': 'june',
 'jul.': 'july',
 'aug.': 'august',
 'sep.': 'september',
 'oct.': 'october',
 'nov.': 'november',
 'dec.': 'december',
 'I’m': 'I am',
 'I’m’a': 'I am about to',
 'I’m’o': 'I am going to',
 'I’ve': 'I have',
 'I’ll': 'I will',
 'I’ll’ve': 'I will have',
 'I’d': 'I would',
 'I’d’ve': 'I would have',
 'amn’t': 'am not',
 'ain’t': 'are not',
 'aren’t': 'are not',
 '’cause': 'because',
 'can’t': 'can not',
 'can’t’ve': 'can not have',
 'could’ve': 'could have',
 'couldn’t': 'could not',
 'couldn’t’ve': 'could not have',
 'daren’t': 'dare not',
 'daresn’t': 'dare not',
 'dasn’t': 'dare not',
 'doesn’t': 'does not',
 'e’er': 'ever',
 'everyone’s': 'everyone is',
 'gon’t': 'go not',
 'hadn’t': 'had not',
 'hadn’t’ve': 'had not have',
 'hasn’t': 'has not',
 'haven’t': 'have not',
 'he’ve': 'he have',
 'he’s': 'he is',
 'he’ll': 'he will',
 'he’ll’ve': 'he will have',
 'he’d': 'he would',
 'he’d’ve': 'he would have',
 'here’s': 'here is',
 'how’re': 'how are',
 'how’d': 'how did',
 'how’d’y': 'how do you',
 'how’s': 'how is',
 'how’ll': 'how will',
 'isn’t': 'is not',
 'it’s': 'it is',
 '’tis': 'it is',
 '’twas': 'it was',
 'it’ll': 'it will',
 'it’ll’ve': 'it will have',
 'it’d': 'it would',
 'it’d’ve': 'it would have',
 'let’s': 'let us',
 'ma’am': 'madam',
 'may’ve': 'may have',
 'mayn’t': 'may not',
 'might’ve': 'might have',
 'mightn’t': 'might not',
 'mightn’t’ve': 'might not have',
 'must’ve': 'must have',
 'mustn’t': 'must not',
 'mustn’t’ve': 'must not have',
 'needn’t': 'need not',
 'needn’t’ve': 'need not have',
 'ne’er': 'never',
 'o’': 'of',
 'o’clock': 'of the clock',
 'ol’': 'old',
 'oughtn’t': 'ought not',
 'oughtn’t’ve': 'ought not have',
 'o’er': 'over',
 'shan’t': 'shall not',
 'sha’n’t': 'shall not',
 'shalln’t': 'shall not',
 'shan’t’ve': 'shall not have',
 'she’s': 'she is',
 'she’ll': 'she will',
 'she’d': 'she would',
 'she’d’ve': 'she would have',
 'should’ve': 'should have',
 'shouldn’t': 'should not',
 'shouldn’t’ve': 'should not have',
 'so’ve': 'so have',
 'so’s': 'so is',
 'somebody’s': 'somebody is',
 'someone’s': 'someone is',
 'something’s': 'something is',
 'that’re': 'that are',
 'that’s': 'that is',
 'that’ll': 'that will',
 'that’d': 'that would',
 'that’d’ve': 'that would have',
 'there’re': 'there are',
 'there’s': 'there is',
 'there’ll': 'there will',
 'there’d': 'there would',
 'there’d’ve': 'there would have',
 'these’re': 'these are',
 'they’re': 'they are',
 'they’ve': 'they have',
 'they’ll': 'they will',
 'they’ll’ve': 'they will have',
 'they’d': 'they would',
 'they’d’ve': 'they would have',
 'this’s': 'this is',
 'those’re': 'those are',
 'to’ve': 'to have',
 'wasn’t': 'was not',
 'we’re': 'we are',
 'we’ve': 'we have',
 'we’ll': 'we will',
 'we’ll’ve': 'we will have',
 'we’d': 'we would',
 'we’d’ve': 'we would have',
 'weren’t': 'were not',
 'what’re': 'what are',
 'what’d': 'what did',
 'what’ve': 'what have',
 'what’s': 'what is',
 'what’ll': 'what will',
 'what’ll’ve': 'what will have',
 'when’ve': 'when have',
 'when’s': 'when is',
 'where’re': 'where are',
 'where’d': 'where did',
 'where’ve': 'where have',
 'where’s': 'where is',
 'which’s': 'which is',
 'who’re': 'who are',
 'who’ve': 'who have',
 'who’s': 'who is',
 'who’ll': 'who will',
 'who’ll’ve': 'who will have',
 'who’d': 'who would',
 'who’d’ve': 'who would have',
 'why’re': 'why are',
 'why’d': 'why did',
 'why’ve': 'why have',
 'why’s': 'why is',
 'will’ve': 'will have',
 'won’t': 'will not',
 'won’t’ve': 'will not have',
 'would’ve': 'would have',
 'wouldn’t': 'would not',
 'wouldn’t’ve': 'would not have',
 'y’all': 'you all',
 'y’all’re': 'you all are',
 'y’all’ve': 'you all have',
 'y’all’d': 'you all would',
 'y’all’d’ve': 'you all would have',
 'you’re': 'you are',
 'you’ve': 'you have',
 'you’ll’ve': 'you shall have',
 'you’ll': 'you will',
 'you’d': 'you would',
 'you’d’ve': 'you would have'}

#Hàm chuyển đổi các từ viết tắt thành các cụm từ chuẩn,không viết tắt
def contraction_fix(word):
    try:
        a=contractions[word]#nếu word là từ viết tắt có trong bộ từ viết tắt => a sẽ là cụm từ không viết tắt của word
    except KeyError:
        a=word # nếu không có key nào trong directory phù hợp với word đã cho=> a sẽ vẫn là word
    return a #trả về từ vựng(Cụm từ vựng không viết tắt)

* **Loại bỏ các chữ số,các kí tự đặc biệt**

In [ ]:
###Loại bỏ các chữ số,các kí tự đặc biệt

def Preprocess(doc): #Hàm loại bỏ các chữ số,các kí tự đặc biệt
    corpus=[]
    for text in tqdm(doc):
        text=" ".join([contraction_fix(w) for w in text.split()])   #tách các từ,thay thế các từ viết tắt bằng các từ đúng,sau đó nối chúng lại
        
        #re là một module để xác định biểu thức chính quy(là một đoạn các ký tự đặc biệt dùng để so khớp các chuỗi hoặc một tập các chuỗi)
        text=re.sub(r'[^a-z0-9A-Z]'," ",text)#Loại bỏ các dấu như !,?.... thay bằng các ' '
        text=re.sub(r'[0-9]{1}',"#",text)#Loại bỏ các số,thay bằng các '#'
        text=re.sub(r'[0-9]{2}','##',text)
        text=re.sub(r'[0-9]{3}','###',text)
        text=re.sub(r'[0-9]{4}','####',text)
        text=re.sub(r'[0-9]{5,}','#####',text)
        corpus.append(text) #thêm dòng vừa rồi vào mảng corpus
    
    return corpus #trả về mảng các câu đã được lược bỏ số và các kí tự đặc biệt

> # -Tạo các vector feature từ dữ liệu
> 
> Khi đã xử lý qua được dữ liệu,cần tiếp tục chuyển đổi dữ liệu vừa được xử lý đó sang dữ liệu có thể training cho model
> > * Lấy ra từ vựng cho các câu
> > * Kiến tạo vector cho các dữ liệu

* **Lấy ra từ vựng cho các câu**


In [ ]:
###Sau khi đã có hàm tiền xử lý cần thiết,tạo hàm lấy ra vốn từ vựng

def get_vocab(corpus):
    vocab={}#Đây là vốn từ vựng của chúng ta(Directory)
    for text in tqdm(corpus): #Lặp qua các câu trong danh sách câu đã được xử lý
        for word in text.split(): #Lặp qua các từ trong các câu
            try:
                vocab[word]+=1 #Nếu từ đó đã có rồi,+1 thêm vào số lượng của từ đó
            except KeyError:
                vocab[word]=1 #Nếu chưa có từ đó,tạo ra 1 key = word và value = 1 mới trong vocab(Directory)
    vocab=dict(sorted(vocab.items(),reverse=True ,key=lambda item: item[1]))#Sắp kếp lại các Key
    return vocab

    Thử hàm tiền xử lý

In [ ]:
#chạy hàm loại bỏ kí tự đặc biệt phía trên
train_processed_doc = Preprocess(train.question_text)
val_processed_doc = Preprocess(val.question_text)

In [ ]:
for i in range(0,10,1):
    randIndex = random.randrange(0,train.question_text.size,1)
    print("Raw:" + train.question_text[randIndex])
    #in ra question ban đầu
    print("Process:" + train_processed_doc[randIndex])
    #in ra hàm tiền sử lý

    Hàm tiền xử lý đã ổn

    Tiến hành lấy ra các từ vựng

In [ ]:
vocabulary = get_vocab(train_processed_doc)

In [ ]:
#Test thử lấy độ dài của danh sách vốn từ
len(vocabulary)

 có 199881 từ

In [ ]:
#vocabulary.items()
print("Bỏ comment nếu cần xem qua các item")

* **Tạo vector**
    * Xử lý theo cách nguyên thủy:

        * Tạo ra một mảng có kích thước bằng số lượng từ vựng để chứa dữ liệu

In [ ]:
##Import
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from scipy.sparse import coo_matrix
from sklearn.metrics import accuracy_score 

In [ ]:
##thử xử lý dữ liệu theo kiểu nguyên thủy nhất
def naiveProcessData(raw_data):
    processedData = []
    for data in tqdm(raw_data):
        #tạo mới directory với các key là vocabulary và value = 0;
        processedSentence = vocabulary.copy()
        for key in dictOfVocabulary.keys():
            processedSentence[key] = 0
        
        #cộng 1 với mỗi word có trong câu.
        for word in data.split():
            try:
                processedSentence[word] += 1;    
            except:
                print(word + "is not in bag of word")
        processedData.append(np.array(processedSentence.values()))
    return processedData

* **Tạo vector**

    * Xử lý theo ma trận thưa


In [ ]:
###Tạo hàm khởi tạo các giá trị cần thiết cho ma trận thưa

#đánh dấu số thứ tự vị trí của các từ trước
index_Vocabulary = vocabulary.copy()
def initIndexVocabulary(indexVocabulary):
    i = 0;
    for key in indexVocabulary.keys():
        indexVocabulary[key] = i
        i += 1

In [ ]:
initIndexVocabulary(index_Vocabulary)
#print(index_Vocabulary.items())

In [ ]:
##Tạo một mảng dữ liệu 3 giá trị như sau: Hàng(dòng dữ liệu thứ mấy),cột(từ đó là từ thứ bao nhiêu trong vocabulary),giá trị(từ đó xuất hiện bao nhiêu lần)
def createConfigValueForMatixSprase(doc):
    rows = []
    columns = []
    values = []
    
    indexSentence = 0; # index của dữ liệu câu hỏi
    for sentence in tqdm(doc): #lặp qua từng câu hỏi trong kho dữ liệu
        dictOfWord = {}  #Khởi tạo tập hợp các từ có trong câu
        for word in sentence.split(): #Xét các từ bên trong một câu
            try: # Nếu tập hợp đã có từ đang xét,số lượng từ đó trong dict tăng lên 1
                dictOfWord[word] += 1;
            except: # Nếu tập hợp chưa có từ đang xét,thêm từ đó vào trong dict,với số lượng từ bằng 1
                dictOfWord.update({word:1})
        #Lặp qua tất cả các từ có trong từ điển của câu.
        for word in dictOfWord.keys():
            rows.append(indexSentence) #Thêm dữ liệu thứ tự dòng
            try:
                columns.append(index_Vocabulary[word]) #Thêm dữ liệu thứ thự cột(Vị trí của từ trong index_Vocabulary)
            except:
                columns.append(len(index_Vocabulary)) # Nếu từ đó không có trong vocabulary,cho nó xuống hang cuối cùng
            values.append(dictOfWord[word]) # Thêm dữ liệu về số lần xuất hiện của từ đó
        indexSentence += 1;
    return (rows,columns,values)

In [ ]:
##Tạo ma trận thưa 
def createSparseMatrix(data):
    rows,columns,values = createConfigValueForMatixSprase(data)
    return coo_matrix((values,(rows,columns)),shape=(len(data),len(index_Vocabulary) + 1))

In [ ]:
#Khởi tạo ma trận traning
train_X = createSparseMatrix(train_processed_doc) #tạo ma trận thưa cho tập train
label_X = train.target

In [ ]:
test_y = createSparseMatrix(val_processed_doc) #tạo ma trận thưa cho tập validation
valid_y = val.target

# Training Model
> * Chọn model
> * Training và thử nghiệm

> # -Chọn model

> đây là bài toán phân loại và là bài toán về ngôn ngữ,nên sử dụng mô hình Naive Bayes Classifiers.

> Việc xử lý dữ liệu đầu vào cho model đã được giải quyết bằng cách xử dụng ma trận thưa ở trên phần tạo vector feature cho dữ liệu

> # -Training Model

* **Sử dụng MultinomialNB**

In [ ]:
#Training Model MultinomialNB()
multi_NB_model = MultinomialNB()
multi_NB_model.fit(train_X,label_X)

* **Tạo các hàm thử đánh giá**

In [ ]:
#Đánh giá model qua tập validation
def validModel(Model):
    pred_y = Model.predict(test_y)
    print('Training size = %d, accuracy = %.2f%%' % \
          (train_X.shape[0],accuracy_score(valid_y, pred_y)*100))

In [ ]:
def compareModel(Models):
    indexRand = random.randrange(0,len(val.target) - 20,1) #chọn random 1 giá trị
    test_dataframe = val[indexRand:indexRand + 10] #lấy 10 giá trị liên tiếp từ số mới random được
    test_data = test_dataframe.question_text
    processed_test_data = Preprocess(test_data) #gọi hàm loại bỏ các kí tự đặc biệt
    test_data_X = createSparseMatrix(processed_test_data) #tạo ma trận thưa với giữ liệu vừa được xử lý
    
    
    for model in Models:
        y_valid_test = test_dataframe.target
        y_pred_test = model.predict(test_data_X)
        print(type(model).__name__[0:4]+":" + str(list(y_pred_test)))
    print("test:" + str(list(y_valid_test))) #in ra kết quả test

In [ ]:
#Hàm thử nghiệm 1 chút
def testModel(Model):
    indexRand = random.randrange(0,len(val.target) - 20,1) #random lấy 1 giá trị
    test_dataframe = val[indexRand:indexRand + 10] #lấy 10 giá trị liên tiếp từ số mới random được
    test_data = test_dataframe.question_text
    processed_test_data = Preprocess(test_data) #gọi hàm loại bỏ các kí tự đặc biệt
    test_data_X = createSparseMatrix(processed_test_data) #tạo ma trận thưa với giữ liệu vừa được xử lý


    y_valid_test = test_dataframe.target
    y_pred_test = Model.predict(test_data_X)
    print(test_dataframe)
    print("test:" + str(list(y_valid_test))) #in ra kết quả test
    print("pred:" + str(list(y_pred_test))) #in ra kết quả thực tế

* **Thử nghiệm độ chính xác của model thông qua validation**

In [ ]:
#đánh giá MultinomialNB()
validModel(multi_NB_model)

    Độ chính xác hơn 92%

In [ ]:
testModel(multi_NB_model)

    Thử nghiệm khá là chuẩn

* **sử dụng BernoulliNB**

In [ ]:
ber_NB_model = BernoulliNB()
ber_NB_model.fit(train_X,label_X)

In [ ]:
validModel(ber_NB_model)

    Độ chính xác hơi tốt hơn một chút

In [ ]:
testModel(ber_NB_model)

> # -Training model với toàn bộ dữ liệu train
chọn ber_NB_model vì chính xác hơn 1 chút

In [ ]:
all_train_preprocess_data = Preprocess(train_data.question_text)
all_train_x = createSparseMatrix(all_train_preprocess_data)
all_train_y = raw_train_data.target
ber_NB_model.fit(all_train_x,all_train_y)

In [ ]:
validModel(ber_NB_model)

train toàn bộ thì được kết quả tốt hơn chút nữa

> # -Vector hóa dữ liệu cần predict

In [ ]:
raw_test_data.head()
preprocessTestData = Preprocess(raw_test_data.question_text) #lấy tập đã loại bỏ hết dữ liệu
testt_padded = createSparseMatrix(preprocessTestData) #đưa dữ liệu vừa lấy được thành ma trận thưa

> # -Predict dữ liệu

In [ ]:
y_test_pre = ber_NB_model.predict(testt_padded)
#print(y_test_pre)